In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow.keras as keras

: 

In [41]:
def createData(df, time_steps, x, y, first, end):
    n = df.shape[0]
    df = df.iloc(0)
    i = 0
    while i < n-1:
        a = []
        b = []
        color = df[i]['Annotation']
        for j in range(time_steps):
            if i >= n-1 or (j > 0 and df[i]['Time'] - df[i-1]['Time'] > 17):
                break
            a.append(list(df[i][first:end]))
            if (color != df[i]['Annotation']):
                break
            i += 1

        if (len(a) != time_steps):
            continue
        x.append(a)
        y.append(color)


    return x, y


def evenOut(x, y):
    #majority of our data consists of y=0, we want to even the data out so that there are equal amounts of 0s and 1s

    zeros = []
    ones = []

    for i in range(len(x)):
        if (y[i] == 0):
            zeros.append(x[i])
        else:
            ones.append(x[i])


    random.shuffle(zeros)

    if (len(zeros) > len(ones)):
        zeros = zeros[:len(ones)]
        

    x = []
    y = []

    for i in range(len(zeros)):
        x.append(zeros[i])
        y.append(0)
        x.append(ones[i])
        y.append(1)

    return x, y

    
def multiDataFrame(text_files, time_steps, first, end):
    x = []
    y = []
    for files in text_files:
        df = pd.read_csv("dataset/" + files, sep=" ", header=0, names=["Time", "AHF", "AV", "AHL", "UHF", "UV", "UHL", "THF", "TV", "THL", "Annotation"])
        df = df[df.Annotation != 0]
        df.Annotation -= 1
        x, y = createData(df, time_steps, x, y, first, end)
    
    x, y = evenOut(x, y)

    return np.array(x), np.array(y)



In [17]:
import os
dir_list = os.listdir("dataset")
dir_list

['S10R01.txt',
 'S02R01.txt',
 'S02R02.txt',
 'S06R01.txt',
 'S06R02.txt',
 'S08R01.txt',
 'S04R01.txt',
 'S01R01.txt',
 'S01R02.txt',
 'S03R02.txt',
 'S03R03.txt',
 'S03R01.txt',
 'S07R02.txt',
 'S07R01.txt',
 'S09R01.txt',
 'S05R01.txt',
 'S05R02.txt']

In [32]:
x.shape, y.shape

((702, 10, 3), (702,))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=12)

val_n = 30

X_val = X_train[-val_n:]
y_val = y_train[-val_n:]
X_train = X_train[:-val_n]
y_train = y_train[:-val_n]

In [7]:
model = keras.Sequential()

model.add(keras.layers.LSTM(7, input_shape=X_train.shape[1:]))
# model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Dense(5, activation="relu"))

model.add(keras.layers.Dense(5, activation="relu"))

model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

2022-12-20 15:02:19.187447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7)                 308       
                                                                 
 dense (Dense)               (None, 5)                 40        
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 384
Trainable params: 384
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['acc', f1_m, precision_m, recall_m])

In [10]:
history = model.fit(X_train, y_train, batch_size=7, epochs=11, validation_data=(X_val, y_val))

Epoch 1/11
194/194 [==============================] - 5s 17ms/step - loss: 0.6855 - acc: 0.6100 - f1_m: 0.6630 - precision_m: 0.5782 - recall_m: 0.8388 - val_loss: 0.6799 - val_acc: 0.6000 - val_f1_m: 0.7311 - val_precision_m: 0.6724 - val_recall_m: 0.9100
Epoch 2/11
194/194 [==============================] - 3s 15ms/step - loss: 0.6710 - acc: 0.6115 - f1_m: 0.6296 - precision_m: 0.5845 - recall_m: 0.7725 - val_loss: 0.6776 - val_acc: 0.6000 - val_f1_m: 0.7111 - val_precision_m: 0.6667 - val_recall_m: 0.8600
Epoch 3/11
194/194 [==============================] - 3s 15ms/step - loss: 0.6512 - acc: 0.6551 - f1_m: 0.6684 - precision_m: 0.6152 - recall_m: 0.8070 - val_loss: 0.6579 - val_acc: 0.6667 - val_f1_m: 0.7800 - val_precision_m: 0.6857 - val_recall_m: 1.0000
Epoch 4/11
194/194 [==============================] - 3s 15ms/step - loss: 0.6568 - acc: 0.6256 - f1_m: 0.6216 - precision_m: 0.6152 - recall_m: 0.7117 - val_loss: 0.6695 - val_acc: 0.5667 - val_f1_m: 0.6238 - val_precision_m: 0.

In [11]:
loss, accuracy, f1_m_value, precision_value, recall_m_value = model.evaluate(X_test, y_test)


19/19 [==============================] - 0s 4ms/step - loss: 0.5686 - acc: 0.7054 - f1_m: 0.7313 - precision_m: 0.6649 - recall_m: 0.8232


In [12]:
print(loss, accuracy, f1_m_value, precision_value, recall_m_value)

0.5686197280883789 0.7053872346878052 0.7313032150268555 0.6649149656295776 0.8231832385063171


In [ ]:
def MODEL(x, y, metrics=['acc', f1_m, precision_m, recall_m], optimizer="Adam", loss="binary_crossentropy", batch_size=20, epochs=25):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=12)

    val_n = 30

    X_val = X_train[-val_n:]
    y_val = y_train[-val_n:]
    X_train = X_train[:-val_n]
    y_train = y_train[:-val_n]


    model = keras.Sequential()

    model.add(keras.layers.LSTM(7, input_shape=X_train.shape[1:]))
    # model.add(keras.layers.Dropout(0.1))

    model.add(keras.layers.Dense(5, activation="relu"))

    model.add(keras.layers.Dense(5, activation="relu"))
    model.add(keras.layers.Dropout(0.1))


    model.add(keras.layers.Dense(1, activation="sigmoid"))

    model.summary()
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=2)

    



    return history, [loss, accuracy, f1_score, precision, recall], model

: 

In [39]:
x, y = multiDataFrame(dir_list, 100, "UHF", "UHL")

In [48]:
history = []
for i in range(1, 5):
    history.append(MODEL(x, y, epochs=(i * 25)))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 7)                 308       
                                                                 
 dense_15 (Dense)            (None, 5)                 40        
                                                                 
 dense_16 (Dense)            (None, 5)                 30        
                                                                 
 dropout_4 (Dropout)         (None, 5)                 0         
                                                                 
 dense_17 (Dense)            (None, 1)                 6         
                                                                 
Total params: 384
Trainable params: 384
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
68/68 [==============================] - 3s 2

In [53]:
for i in history:
    print(i[1][1])

0.6531986594200134
0.7171717286109924
0.6498316526412964
0.6902356743812561


In [54]:
x, y = multiDataFrame(dir_list, 100, "AHF", "THL")

In [55]:

MODEL(x, y, epochs=(i * 25))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 7)                 476       
                                                                 
 dense_27 (Dense)            (None, 5)                 40        
                                                                 
 dense_28 (Dense)            (None, 5)                 30        
                                                                 
 dropout_8 (Dropout)         (None, 5)                 0         
                                                                 
 dense_29 (Dense)            (None, 1)                 6         
                                                                 
Total params: 552
Trainable params: 552
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
68/68 [==============================] - 3s 2

0.6144781112670898
0.7255892157554626
0.68855220079422
0.6936026811599731
